In [3]:
import sys

import numpy as np
import pandas as pd
import s3fs
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.analysis.make_analysis_data as ad
import entropy.data.cleaners as cl
import entropy.data.creators as cr
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

In [4]:
dfr = hd.read_raw_data("777")
hd.inspect(dfr)

Time for read_raw_data                 : 4.45 seconds
(682,656, 27)


,Transaction Reference,User Reference,User Registration Date,Year of Birth,Salary Range,Postcode,LSOA,MSOA,Derived Gender,Transaction Date,Account Reference,Provider Group Name,Account Type,Latest Recorded Balance,Transaction Description,Credit Debit,Amount,User Precedence Tag Name,Manual Tag Name,Auto Purpose Tag Name,Merchant Name,Merchant Business Line,Account Created Date,Account Last Refreshed,Data Warehouse Date Created,Data Warehouse Date Last Updated,Transaction Updated Flag
0,688293,777,2011-07-20,1969.0,20K to 30K,WA1 4,E01012553,E02002603,M,2012-01-25,262916,NatWest Bank,Current,364.220001,"9572 24jan12 , tcs bowdon , bowdon gb - pos",Debit,25.030001,No Tag,No Tag,No Tag,No Merchant,Unknown Merchant,2011-07-20,2020-07-21 20:32:00,2014-07-18,2017-10-24,U
1,688291,777,2011-07-20,1969.0,20K to 30K,WA1 4,E01012553,E02002603,M,2012-01-25,262916,NatWest Bank,Current,364.220001,warrington b.c. - d/d,Debit,26.000000,Gym Membership,Gym Membership,No Tag,No Merchant,Unknown Merchant,2011-07-20,2020-07-21 20:32:00,2014-07-18,2017-10-23,U


In [5]:
dft = hd.read_txn_data("777")
hd.inspect(dft)

Time for read_txn_data                 : 3.59 seconds
(649,820, 24)


,id,date,user_id,amount,desc,merchant,tag_group,tag,user_registration_date,account_created,account_id,account_last_refreshed,account_provider,account_type,debit,desco,female,latest_balance,merchant_business_line,postcode,tag_auto,tag_manual,tag_up,yob
0,688300,2012-02-01,777,400.00,mdbremoved,NaN,transfers,other_transfers,2011-07-20,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,True,<mdbremoved> - s/o,0.0,364.220001,non merchant mbl,wa1 4,transfers,other account,other account,1969.0
1,688299,2012-02-01,777,3.03,aviva pa,aviva,spend,finance,2011-07-20,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,True,aviva pa - d/d,0.0,364.220001,aviva,wa1 4,health insurance,life insurance,life insurance,1969.0


In [20]:
dft

,id,date,user_id,amount,desc,merchant,tag_group,tag,user_registration_date,account_created,account_id,account_last_refreshed,account_provider,account_type,debit,desco,female,latest_balance,merchant_business_line,postcode,tag_auto,tag_manual,tag_up,yob
0,688300,2012-02-01,777,400.000000,mdbremoved,NaN,transfers,other_transfers,2011-07-20,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,True,<mdbremoved> - s/o,0.0,364.220001,non merchant mbl,wa1 4,transfers,other account,other account,1969.0
1,688299,2012-02-01,777,3.030000,aviva pa,aviva,spend,finance,2011-07-20,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,True,aviva pa - d/d,0.0,364.220001,aviva,wa1 4,health insurance,life insurance,life insurance,1969.0
2,688301,2012-02-03,777,8.750000,chart ins log tran,NaN,NaN,NaN,2011-07-20,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,True,chart ins log tran - d/d,0.0,364.220001,NaN,wa1 4,NaN,memberships,memberships,1969.0
3,688303,2012-02-03,777,0.990000,9572 02feb 12 apple itunes store gbp london gb,apple,spend,services,2011-07-20,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,True,"9572 02feb12 , apple itunes store, gbp , london gb - pos",0.0,364.220001,apple,wa1 4,"entertainment, tv, media",NaN,mobile app,1969.0
4,688302,2012-02-03,777,20.000000,9572 02feb 12 national lottery inte watford gb,camelot,spend,services,2011-07-20,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,True,"9572 02feb12 , national lottery , inte , watford gb - pos",0.0,364.220001,camelot,wa1 4,gambling,NaN,gambling,1969.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649815,802522114,2020-07-28,587777,-95.000000,mdbremoved bgc,NaN,spend,retail,2020-06-19,2020-06-19,1711781,2020-08-16 18:36:00,barclays,current,False,<mdbremoved> bgc,0.0,5699.700195,NaN,w7 3,refunded purchase,NaN,refunded purchase,1973.0
649816,802887369,2020-07-28,587777,5.400000,tfl travel charge tfl gov uk cp,tfl,spend,travel,2020-06-19,2020-06-19,1711784,2020-08-16 20:35:00,american express,credit card,True,tfl travel charge tfl.gov.uk/cp,0.0,-1844.410034,tfl,w7 3,public transport,NaN,public transport,1973.0
649817,802887368,2020-07-28,587777,10.650000,marks spencer plc st london,marks and spencer,spend,household,2020-06-19,2020-06-19,1711784,2020-08-16 20:35:00,american express,credit card,True,marks & spencer plc st london,0.0,-1844.410034,marks & spencer dept store,w7 3,"food, groceries, household",NaN,"food, groceries, household",1973.0
649818,803052582,2020-07-31,587777,3.320000,sanef sa stl 404 ddr,NaN,NaN,NaN,2020-06-19,2020-06-19,1711781,2020-08-16 18:36:00,barclays,current,True,sanef sa stlxxxxx404 ddr,0.0,5699.700195,NaN,w7 3,NaN,NaN,NaN,1973.0


In [44]:
month = pd.Grouper(key="date", freq="m")
idx_cols = ["user_id", month]


def income(df):
    """Yearly income.

    Calculated yearly incomes are scaled to 12-month incomes to account for
    user-years with incomplete data and multiplied by -1 to get positive
    numbers (credits are negative in dataset).
    """
    year = pd.Grouper(key="date", freq="year")
    mask = df.tag_group.eq("income") & ~df.debit
    yearly_incomes = (
        df.loc[mask]
        .groupby(["user_id", year])
        .agg(income=("amount", "sum"), observed_months=("ym", "nunique"))
        .assign(income=lambda df: df.income / df.observed_months * -12)
        .drop(columns="observed_months")
    )
    return df.merge(
        yearly_incomes,
        left_on=["user_id", year],
        right_on=["user_id", "year"],
        validate="m:1",
    ).drop(columns="year")


def income(df):

    is_income_pmt = df.tag_group.eq("income") & ~df.debit
    income_pmts = df.amount.where(is_income_pmt, np.nan) * -1

    return income_pmts.groupby([df.user_id, df.date.dt.to_period("m")]).sum()


income(dft)

user_id  date   
777      2012-02    1901.329956
         2012-04    1909.969971
         2012-05    1909.969971
         2012-06    1909.770020
         2012-07    1935.369995
                       ...     
587777   2020-03      82.800003
         2020-04      83.849998
         2020-05      84.199997
         2020-06      84.199997
         2020-07       0.000000
Name: amount, Length: 7578, dtype: float32